In [1]:
import pandas as pd

In [2]:
sequences = pd.read_csv("../data/ml-1m/raw_sequences.csv")

In [3]:
sequences

,user_id,movie_id
0,1,1193 661 914 3408 2355 1197 1287 2804 594 919 ...
1,2,1357 3068 1537 647 2194 648 2268 2628 1103 291...
2,3,3421 1641 648 1394 3534 104 2735 1210 1431 386...
3,4,3468 1210 2951 1214 1036 260 2028 480 1196 119...
4,5,2987 2333 1175 39 288 2337 1535 1392 2268 1466...
...,...,...
6035,6036,571 574 2053 2054 2058 588 589 4 3005 6 3006 3...
6036,6037,589 3006 1407 2064 2065 593 3015 903 904 905 9...
6037,6038,1419 920 3088 232 1136 1148 1183 2146 3548 356...
6038,6039,588 2067 1416 3022 3028 2080 2083 2087 3035 30...


In [64]:
# テストデータに含まれる顧客の学習データが何個かあるかによって学習データを分けたい
# 学習データがx個ある顧客のみを含んだテストデータ
# x = [10, 20, 30, 40, 50]
sequences.movie_id = sequences.movie_id.apply(lambda s: s.split(" "))

In [119]:
# 各系列ごとの末尾n個をテストデータとする

import random
from typing import Dict, List, Tuple
random.seed(0)

a = []

counter = {}
for i in range(5 + 1):
    counter[i * 10] = 0

for _, (user_id, movie_id) in sequences.iterrows():
    seq = movie_id
    seq_l = len(seq)

    while True:
        l = random.randint(0, 5)
        if l == 0:
            train_seq = seq
            test_seq = []
            train_l = 0
            counter[train_l] += 1
            a.append((user_id, train_l, train_seq, test_seq))
            break
        else:
            train_l = l * 10
            if counter[train_l] == 500:
                continue

            test_l = 20
            required_l = train_l + test_l
            if seq_l < required_l:
                continue
            train_seq = seq[-required_l:-test_l]
            test_seq = seq[-test_l:]
            counter[train_l] += 1
            a.append((user_id, train_l, train_seq, test_seq))
            break

{0: 3540, 10: 500, 20: 500, 30: 500, 40: 500, 50: 500}

In [141]:
train_sequences = {}
test_sequences = {}
all_test_sequences = {}

test_size = [10, 20, 30, 40, 50]
for sz in test_size:
    test_sequences[sz] = {}

for user_id, train_l, train_seq, test_seq in a:
    assert user_id not in train_sequences
    train_sequences[user_id] = " ".join(train_seq)
    if train_l > 0:
        all_test_sequences[user_id] = " ".join(test_seq)
        test_sequences[train_l][user_id] = " ".join(test_seq)

for sz in test_size:
    file_path = f"../data/ml-1m/test-{sz}.csv"
    test_sz_df = pd.DataFrame.from_dict(test_sequences[sz], orient="index", columns=["sequence"])
    test_sz_df.index.name = "user_id"
    test_sz_df.to_csv(file_path)

In [142]:
train_df = pd.DataFrame.from_dict(train_sequences, orient="index", columns=["sequence"])
train_df.index.name = "user_id"
train_df.to_csv("../data/ml-1m/train.csv")

In [143]:
test_df = pd.DataFrame.from_dict(all_test_sequences, orient="index", columns=["sequence"])
test_df.index.name = "user_id"
test_df.to_csv("../data/ml-1m/test.csv")